In [69]:
import os
import numpy as np
import pandas as pd
import scipy.optimize as opt

In [70]:
class ChoiceModels(object):
    
    '''
    This class defines methods that will be used later in speficying and estimating choice models.
    '''
   
    def load_data(self, path, file):
        df = pd.read_csv(os.path.join(path, file), sep='\s+', header=0)
        df['cons'] = 1.
        return df
    
    def expand_data(self, df, n):
        '''
        Parameters
        ----------
        df : a pandas data frame
            
        n : Integer
            Number of times to expand the data
        Returns
        -------
        An expanded pandas data frame with a panel structure
        '''
        df['Alt'] = [[str(i) for i in range(n)] for _ in range(len(df))]
        return df.explode('Alt')
    
    def create_choice_attributes(self, df, config):
        '''
        This method creates a panel structure of data to estimate the multinomial
        choice model speficied in the configuration file (config-- a json format file)
        '''
        # create dependent variable
        y_namelist = list(config['Alternatives']['0'].keys())
        df['choice'] = list(zip(*[df[v] for v in y_namelist]))
        df = self.expand_data(df, len(config['Alternatives']))
   
        df['y'] = 0.
        for k,v in config['Alternatives'].items():
            label = tuple(v.values())
            df.loc[(df["Alt"]==k) & (df['choice']==label), 'y'] = 1
        
        # create alternative specific attributes
        dic = config['Attributes']
        for var,info in dic.items():
            df[var] = 0
            for alt, w in info['rule'].items():
                df['junk'] = 0
                df.loc[(df['Alt'] == alt), 'junk'] = 1
                df[var] = df[var] + w * df[info['variable']] * df['junk'] 
        df = df.drop("junk", axis='columns')
        
        # creat interactions
        df, xz_list = self.create_interactions(df, config['Interactions']) 
        x_list = list(config['Attributes'].keys()) + xz_list
        return {'data': df, "var_names": x_list}
    
    def create_interactions(self, df, interact_list):
        '''
        Parameters
        ----------
        df : pandas data frame
            
        interact_list : a List
            The list contains pairs of variable names as tuples
        Returns
        -------
        df : pandas data frame after adding interactions
            
        xz_list : A list of created interactions
        '''
        xz_list = []
        for item in interact_list:
            vname = item[0] + "_" + item[1]
            df[vname] = df[item[0]] * df[item[1]]
            xz_list.append(vname)
        return df, xz_list 
        
        
    def optimization(self, objfun, para):
        '''
        Parameters
        ----------
        objfun : a user defined objective function of para
            
        para : a 1-D array with the shape (k,), where k is the number of parameters.
        Returns
        -------
        dict
            A dictionary containing estimation results
        '''
        v = opt.minimize(objfun, x0=para, jac=None, method='BFGS', 
                          options={'maxiter': 1000, 'disp': True})  
        return {'log_likelihood':-1*v.fun, "Coefficients": v.x, "Var_Cov": v.hess_inv}

In [71]:
class BinaryLogit(ChoiceModels):
    '''
    This class is to estimate a binary logit nodel by MLE.  
    '''
    def __init__(self, path, file, yname, x=None, z=None, interactions=None):
        df = super().load_data(path, file)
        if x is None:
            x = []
        if z is None:
            z = []
        if interactions is None:
            xz = []
            self.df = df
        else:
            self.df, xz = super().create_interactions(df, interactions)
            
        self.X_list = ['cons'] + x + z + xz
        self.Xmat = self.df[self.X_list].to_numpy()
        self.y = self.df[yname].to_numpy()
        
    def log_likelihood(self, para):
        '''
        Parameters
        ----------
        para : array
            a 1-D array with the shape(k,), where k is the number of model parameters.
        Returns
        -------
        res : scalar
            log-likelihood value
        '''
        xb = np.matmul(self.Xmat, para)
        xb = np.exp(xb)
        xb = xb / (1+xb)
        return (-1/len(xb)) * np.sum(self.y * np.log(xb) + (1-self.y) * np.log(1 - xb))
   
    def estimation(self, para):
        '''
        Parameters
        ----------
        para : array
            a 1-D array with the shape(k,), where k is the number of model parameters.
        Returns
        -------
        A dictionary of estimation results
        '''
        return super().optimization(self.log_likelihood, para)

In [90]:
class MultinomialLogit(ChoiceModels):

    # Specify model here    
    model_config = {"Alternatives":
                    {"0": { "occupancy": 1, "route": 1},
                     "1": { "occupancy": 1, "route": 0},
                     "2": { "occupancy": 2, "route": 1},
                     "3": { "occupancy": 2, "route": 0},
                     "4": { "occupancy": 3, "route": 1},
                     "5": { "occupancy": 3, "route": 0},
                     
},

                    "Attributes":{
                                  'express_dummy':{'variable':'route', 
                                                   'rule':{"0":1,"2":1,"4":1}},
                                  'hov2_dummy':{'variable':'occupancy', 
                                               'rule':{"2":1,"3":1}},
                                  "hov3_dummy":{'variable':'occupancy', 
                                                'rule':{"4":1,"5":1}},
                                  "price":{"variable": 'toll', 
                                           "rule": {"0":1,"2":1/2,"4":1/6}},
                                  "time": {"variable":"median", 
                                           "rule":{"0":1,"2":1,"4":1}}},
                    "Interactions":[('price', "high_income"), ('price', "med_income"),
                                    ("hov2_dummy", "householdsize"),
                                    ("hov3_dummy", "householdsize")],
                    "Mixedlogit":['price', 'time', 'express_dummy', 'hov2_dummy', 'hov3_dummy']}
    
    #MNL里面的x是什么？
    def __init__(self, path, file):
        df = super().load_data(path, file)
        res = super().create_choice_attributes(df, MultinomialLogit.model_config)
        self.df = res['data']
        self.X_list = res['var_names']
        self.y = self.df['y'].to_numpy()
        self.Xmat = self.df[self.X_list].to_numpy()
               
    def mnl_log_likelihood(self, para):
        '''
        This method defines the data log-likelihood from a Multinomial Logit.
        '''
        df = self.df.copy()
        xb = np.matmul(self.Xmat, para)
        xb = np.exp(xb)
        df['xb'] = xb.tolist()
        # group sum
        df['xbsum'] = df.groupby(['id'])["xb"].transform(lambda x: x.sum())
        df['log_likelihood'] = df['y']*np.log(df['xb'] / df['xbsum'])
        return (-1/len(df))* np.sum(df['log_likelihood'])
    
    def estimation(self, para):
        '''
        Parameters
        ----------
        para : array
            a 1-D array with the shape(k,), where k is the number of model parameters.
        Returns
        -------
        A dictionary of estimation results
        '''
        return super().optimization(self.mnl_log_likelihood, para)

In [106]:
if __name__ == '__main__':
    p = r"/Users/fangyanwang/Desktop/514/Assignment"
    f = "assignment 1.txt"
    ## estimating binary models
    x = ['toll', 'median']
    z = ['female', 'age3050', 'distance', 'householdsize']
    interactions = [('toll', 'high_income'), ('toll', 'med_income')]
    trans = BinaryLogit(p, f, "trans", x=x, z=z, interactions=interactions)
    bini = np.zeros(len(trans.X_list))
    res_binary = trans.estimation(bini)
        ## estimating a MNL model
    
   # x = ['female', 'age3050', 'distance', 'householdsize']
    mnl = MultinomialLogit(p, f)
    bini = np.zeros(len(mnl.X_list))
    res_mnl = mnl.estimation(bini)

<ipython-input-71-030a815f1154>:35: RuntimeWarning: divide by zero encountered in log
  return (-1/len(xb)) * np.sum(self.y * np.log(xb) + (1-self.y) * np.log(1 - xb))
<ipython-input-71-030a815f1154>:35: RuntimeWarning: invalid value encountered in multiply
  return (-1/len(xb)) * np.sum(self.y * np.log(xb) + (1-self.y) * np.log(1 - xb))
<ipython-input-71-030a815f1154>:33: RuntimeWarning: overflow encountered in exp
  xb = np.exp(xb)
<ipython-input-71-030a815f1154>:34: RuntimeWarning: invalid value encountered in true_divide
  xb = xb / (1+xb)
<ipython-input-71-030a815f1154>:35: RuntimeWarning: divide by zero encountered in log
  return (-1/len(xb)) * np.sum(self.y * np.log(xb) + (1-self.y) * np.log(1 - xb))
<ipython-input-71-030a815f1154>:35: RuntimeWarning: invalid value encountered in multiply
  return (-1/len(xb)) * np.sum(self.y * np.log(xb) + (1-self.y) * np.log(1 - xb))
<ipython-input-71-030a815f1154>:33: RuntimeWarning: overflow encountered in exp
  xb = np.exp(xb)
<ipython-inp

         Current function value: nan
         Iterations: 1
         Function evaluations: 1120
         Gradient evaluations: 112
Optimization terminated successfully.
         Current function value: 0.167032
         Iterations: 76
         Function evaluations: 770
         Gradient evaluations: 77


In [101]:
res_binary

{'log_likelihood': nan,
 'Coefficients': array([  34.69067291,   98.13458041, -139.99123025,   23.12711527,
          34.69067291, 1008.29680568,  107.63003467,   65.86780899,
          69.98176311]),
 'Var_Cov': array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1]])}

In [107]:
res_mnl

{'log_likelihood': -0.16703183583113315,
 'Coefficients': array([ 6.05153654e+02, -1.87948549e-01, -9.32984027e-01,  2.27185690e-01,
         7.39959550e+01,  4.30960795e-01,  2.96794325e-01, -6.56813524e-03,
         1.18619217e-01]),
 'Var_Cov': array([[ 1.96474209e+10,  3.47943095e+04,  1.60137218e+05,
         -1.27456705e+05,  2.42462660e+09,  3.22528615e+04,
          2.41886159e+05, -6.20917938e+03, -3.64482357e+04],
        [ 3.47943095e+04,  7.62362964e+01,  2.86289366e+01,
          1.55836954e+01,  4.24686836e+03, -1.05708207e+01,
         -1.15023986e+01, -1.77759913e+01, -6.76574254e+00],
        [ 1.60137218e+05,  2.86289366e+01,  1.07215907e+02,
          3.15362313e+01,  1.97938144e+04, -1.82718244e+01,
         -1.85865368e+01, -6.03159932e+00, -2.16685185e+01],
        [-1.27456705e+05,  1.55836954e+01,  3.15362313e+01,
          9.36603205e+01, -1.59311413e+04, -8.43443706e+01,
         -8.70441522e+01, -1.86722517e+00, -4.57373857e+00],
        [ 2.42462660e+09,  4.

In [98]:
class NestedLogit(ChoiceModels):

    # Specify model here    
    model_config = {"Alternatives":
                    {"0": {"trans": 1, "occupancy": 1, "route": 1},
                     "1": {"trans": 1, "occupancy": 1, "route": 0},
                     "2": {"trans": 1, "occupancy": 2, "route": 1},
                     "3": {"trans": 1, "occupancy": 2, "route": 0},
                     "4": {"trans": 1, "occupancy": 3, "route": 1},
                     "5": {"trans": 1, "occupancy": 3, "route": 0},
                     "6": {"trans": 0, "occupancy": 1, "route": 0},
                     "7": {"trans": 0, "occupancy": 2, "route": 0},
                     "8": {"trans": 0, "occupancy": 3, "route": 0}},
                    "Nests": {"0":{"0": ["0", "1"], "1": ["2", "3"], 
                                   "2": ["4", "5"]},"1":["6", "7", "8"]},
                    "Attributes":{'trans_dummy':{'variable': 'trans', 
                                                 'rule':{"0":1,"1":1,
                                                         "2":1,"3":1,"4":1,"5":1}},
                                  'express_dummy':{'variable':'route', 
                                                   'rule':{"0":1,"2":1,"4":1}},
                                  'hov2_dummy':{'variable':'occupancy', 
                                               'rule':{"2":1,"3":1,"7":1}},
                                  "hov3_dummy":{'variable':'occupancy', 
                                                'rule':{"4":1,"5":1,"8":1}},
                                  "price":{"variable": 'toll', 
                                           "rule": {"0":1,"2":1/2,"4":1/6}},
                                  "time": {"variable":"median", 
                                           "rule":{"0":1,"2":1,"4":1}}},
                    "Interactions":[('price', "high_income"), ('price', "med_income"),
                                    ("hov2_dummy", "householdsize"),
                                    ("hov3_dummy", "householdsize")],
                    "Mixedlogit":['price', 'time', 'trans_dummy', 'hov2_dummy', 'hov3_dummy']}
    
    
    def __init__(self, path, file):
        df = super().load_data(path, file)
        res = super().create_choice_attributes(df, NestedLogit.model_config)
        self.df = res['data']
        self.X_list = res['var_names']
        self.y = self.df['y'].to_numpy()
        self.Xmat = self.df[self.X_list].to_numpy()
 
    def nl_log_likelihood(self, para):
        #beta = para[0:k]
        #lamda = para[k+1:k+2]
        #delta = para[k+3:]
        df = self.df.copy()
        xb = np.matmul(self.Xmat, para)
        xb = np.exp(xb)
        df['xb'] = xb.tolist()
        
        
        # calculate the probability for each nest
        for nest in self.model_config["Nests"]:
            nest_prob = 1
            for alt in self.model_config["Nests"][nest]:
                nest_prob *= df.loc[df["alt"] == alt, "xb"]
            df.loc[df["alt"].isin(self.model_config["Nests"][nest]), "nest_prob"] = nest_prob
        
        # calculate the overall probability for each alternative
        df["overall_prob"] = df["xb"]
        for nest in self.model_config["Nests"]:
            df.loc[df["alt"].isin(self.model_config["Nests"][nest]), "overall_prob"] *= df.loc[df["alt"].isin(self.model_config["Nests"][nest]), "nest_prob"]
        
        # calculate the log-likelihood
        df["log_likelihood"] = df["y"] * np.log(df["overall_prob"] / df["overall_prob"].sum())
        return (-1/len(df)) * np.sum(df["log_likelihood"])
    
            # group sum
        df['xbsum'] = df.groupby(['id'])["xb"].transform(lambda x: x.sum())
        df['log_likelihood'] = df['y']*np.log(df['xb'] / df['xbsum'])
        return (-1/len(df))* np.sum(df['log_likelihood'])
    
    def estimation(self, para):
        '''
        Parameters
        ----------
        para : array
            a 1-D array with the shape(k,), where k is the number of model parameters.
        Returns
        -------
        A dictionary of estimation results
        '''
        return super().optimization(self.nl_log_likelihood, para)

In [103]:
if __name__ == '__main__':
    p = r"/Users/fangyanwang/Desktop/514/Assignment"
    f = "assignment 1.txt"
    
        ## estimating a nested logit model
    x = ['female', 'age3050', 'distance', 'householdsize']
    nl = NestedLogit(p, f)
    bini = np.zeros(len(nl.X_list))
    res_nl = nl.estimation(bini)

KeyError: 'alt'